In [1]:
'''
Based on :

FINAL_LookupTables_US_ITC_HTS_Codes_2015-2017.xlsx
https://docs.google.com/spreadsheets/d/1jAx6VSln3-sCxI7R1TDtAVSZjzVsH36MpAq3yKa4e7o/edit#gid=1901895138

'''
import pandas as pd
import re
import textacy
import spacy
nlp = spacy.load('en')
import math
import gen_kw_2
import gen_kw_2
import filter_hs_codes
import collections

In [2]:

'''
Files
1501_hts_delimited.txt
hts_2016_basic_delimited.csv
hts_2017_preliminary_csv.csv
hts_2018_basic_csv.csv
'''

EXPERIMENT = True

def get_file_list_1():
    global EXPERIMENT
    
    if EXPERIMENT == True:
        files = {
            '1501_hts_delimited.txt': 'hs_codes_file_1_1.csv',
            'hts_2016_basic_delimited.csv': 'hs_codes_file_2_1.csv',
            'hts_2017_preliminary_csv.csv': 'hs_codes_file_3_1.csv',
#             'hts_2018_basic_csv.csv': 'hs_codes_file_4_1.csv'
        }
    return files

    files = {
        '1501_hts_delimited.txt': 'hs_codes_file_1.csv',
        'hts_2016_basic_delimited.csv': 'hs_codes_file_2.csv',
        'hts_2017_preliminary_csv.csv': 'hs_codes_file_3.csv',
        'hts_2018_basic_csv.csv': 'hs_codes_file_4.csv',
    }
    return files

def get_file_list_2():
    global EXPERIMENT
    files = [
        'hs_codes_file_1.csv',
        'hs_codes_file_2.csv',
        'hs_codes_file_3.csv',
        'hs_codes_file_4.csv'
    ]
    
    if EXPERIMENT == True:
        files = [
            'hs_codes_file_1_1.csv',
            'hs_codes_file_2_1.csv',
            'hs_codes_file_3_1.csv'
#             'hs_codes_file_4_1.csv'
        ]
    return files
    return files

# ---------------------------- #

In [28]:
def process_chg_file(input_file, op_file):
    parts = input_file.split('.')
    file_ext = parts[1]

    if file_ext == 'txt':
        df = pd.read_csv(input_file, sep='|', encoding="ISO-8859-1")
    else:
        df = pd.read_csv(input_file, encoding="ISO-8859-1")

    def clean_description(row):
        if row is None or row['Description'] is None:
            return None
        text = str(row['Description'])
        if len(text) == 0:
            return None
#         pattern = re.compile(r'<[^>]+>')
#         text = pattern.sub('', text)
        strip_chars = ['\t', ' ', '\t\n', ':', ',', ';']
        for s in strip_chars:
            text = text.strip(s)
        text = text.replace(',', ' && ')
        text = text.replace(' and ', ' && ')
        return text

    op_path = op_file
    df['Description'] = df.apply(clean_description, 1)
    delete_cols = [
        'Unit of Quantity',
        'General Rate of Duty',
        'Special Rate of Duty',
        'Column 2 Rate of Duty',
        'Col. 1 Rate',
        'Special Rate',
        'Col. 2 Rate',
        'Footnote / Comment',
        'Unnamed: 2'
    ]

    for d in delete_cols:
        try:
            del df[d]
        except:
            pass
    # in case of 'HTS No.', 'Stat Suffix'  - combine them into 'hs_code'
    # also rename Level as Indent

    try:
        df = df.rename(columns={'Level': 'Indent'})
    except:
        pass

    def combine_hscode(row):
        h1 = row['HTS No.']
        h2 = row['Stat Suffix']
        if math.isnan(h2) == False and (int(h2)) > 0:
            res = h1 + '.' + str(int(h2))
        else:
            res = h1
        return res

    df_col_names = list(df.columns)

    if ('HTS No.' in df_col_names) and ('Stat Suffix' in df_col_names):
        df['hs_code'] = df.apply(combine_hscode, axis=1)
        del df['HTS No.']
        del df['Stat Suffix']
    else:
        try:
            df = df.rename(columns={'HTS Number': 'hs_code'})
        except:
            pass

    print(df.columns)
    df = filter_hs_codes.filter_df(df,level =1 )
    df.to_csv(op_path)
    print('-----------------')
    return

# ----------------------------------- #

# Clean and parse from txt/csv to csv. Place files in csv with new names

def main_1():
    files = get_file_list_1()
    for inp, op in files.items():
        process_chg_file(inp, op)

# ----------------------------------- #

main_1()

Index(['Indent', 'Description', 'hs_code'], dtype='object')
-----------------
Index(['hs_code', 'Indent', 'Description'], dtype='object')
-----------------
Index(['hs_code', 'Indent', 'Description'], dtype='object')
-----------------


Extract scientific names
------

In [29]:
def get_sc_names(file):
    df = pd.read_csv(file)
    df['sc_names'] = None
    for i,row in df.iterrows():
        desc_text = row['Description']
        if desc_text is None or type(desc_text) != str:
            continue
        res_list = [] 
        patterns = [
            '\<\S+\>[A-Z][a-z]+\s[a-z]*\<\S+\>',
            '\<\S+\>[A-Z][a-z]+\s*\<\S+\> [a-z]+\.',
            '\<\S+\>[A-Z][a-z]+\<\S+\> \<\S+\>[a-z]+\<\S+\>'
        ]
        
        for pattern in patterns :
            search_res = re.findall(pattern,desc_text)
            if len(search_res)> 0 :
#                 print (' -->' , desc_text)
#                 print (search_res)
                for grp in range(len(search_res)) :
                    _res = search_res[grp]
                    _res = ''.join(_res)
                    _pattern = re.compile(r'<[^>]+>')
                    text = _pattern.sub('', _res)
                    text = text.strip()
                    res_list.append(text)
               
        if len(res_list)>0 :
            res = ';'.join(res_list)
            df.loc[i,'sc_names'] = res
        pattern = re.compile(r'<[^>]+>')
        text = pattern.sub('', desc_text)   
        df.loc[i,'Description'] = text
        
    df.to_csv(file)
    return
            
def main_2():
    files = get_file_list_2()
    for inp in files :
        get_sc_names(inp)
        
# ----------------------------------- #  


main_2()

In [30]:
def collate_sc_names(file):
    df = pd.read_csv(file)
    
    def gather_sc_names(df, idx):
        cur_row = df.loc[idx]
        cur_indent = cur_row['Indent']
        indent_attr = 'Indent'
        min_idx = 0
        res =  cur_row['sc_names']
        if type(res)== str :
            res = res.split(';')
        else:
            res = []
        cur_idx = idx
        
        if cur_indent == 0:
            return  cur_row['sc_names']
        else:
            # go up and fetch
            cur_idx = idx - 1
            
            while cur_idx >= min_idx:
                r = df.loc[cur_idx]
                if r[indent_attr] == cur_indent - 1:
                    scn = r['sc_names']
                    if type(scn) == str :
                        scn = scn.split(';')
                        res.extend(scn)
                        res = list(set(res))
                    break
                cur_idx -= 1
        if isinstance(res, collections.Iterable) :
            tmp = []
            for r in res:
                if r != '':
                    tmp.append(r)
            res = ';'.join(tmp)
        return res

        
    for i,row in df.iterrows():
        sc_names = gather_sc_names(df, i)
        df.at[i, 'sc_names'] = sc_names
    df.to_csv(file)
        

def main_5():
    files = get_file_list_2()
    for inp in files :
        collate_sc_names(inp)

main_5()  

Write out the complete descriptions
------

In [31]:
def main_3_aux(file):

    def gen_aux_desc(df, idx):
        min_idx = 0
        neg_keyword = 'Other'
        res = None
        cur_row = df.loc[idx]
        indent_attr = 'Indent'
        try:
            cur_indent = cur_row['Indent']
        except:
            cur_indent = cur_row['Level']
            indent_attr = 'Level'

        cur_desc = cur_row['Description']
        is_neg_keyword = False

        if type(cur_desc) != str:
            cur_desc = ''

        if cur_desc == neg_keyword:
            is_neg_keyword = True

        def format_desc(s):
            return '[ ' + s + ' ]'

        if cur_indent == 0:
            res = format_desc(cur_desc)
        else:
            # go up and fetch
            cur_idx = idx - 1
            if is_neg_keyword:
                res_parts = []
                while cur_idx >= min_idx:
                    r = df.loc[cur_idx]
                    # Find entry in same level
                    if r[indent_attr] == cur_indent:
                        # add in what the thing is not
                        d3 = r['Description']
                        if type(d3) == str and len(d3) > 0:
                            t = '!' + format_desc(d3)
                            res_parts.append(t)

                    elif r[indent_attr] == cur_indent - 1:
                        fd = r['full_desc']
                        res = ' && '.join(res_parts)
                        res = format_desc(res)
                        res = fd + ' && ' + res
                        res = format_desc(res)
                        break

                    cur_idx -= 1

            else:
                while cur_idx >= min_idx:
                    r = df.loc[cur_idx]
                    dsc = r['full_desc']
                    if r[indent_attr] == cur_indent - 1:
                        res = dsc + ' && ' + format_desc(cur_desc)
                        break
                    else:
                        cur_idx -= 1
        return res

    def generate_desc(df):
        df.at[:, 'full_desc'] = None
        new_df = df.copy(deep=True)
        for i, row in df.iterrows():
            _desc = gen_aux_desc(new_df, i)
            new_df.at[i, 'full_desc'] = _desc
        return new_df




    df = pd.read_csv(file,index_col=0)
    df = generate_desc(df)
    try:
        del df['Unnamed: 0']
    except:
        pass
    print('main_3_aux', file)
    print('-----------------')
    df.to_csv(file)

    
def main_3():
    files = get_file_list_2()
    for inp in files :
        main_3_aux(inp)
        
        
main_3()


main_3_aux hs_codes_file_1_1.csv
-----------------
main_3_aux hs_codes_file_2_1.csv
-----------------
main_3_aux hs_codes_file_3_1.csv
-----------------


Add in keywords - unigrams, bi-grams and trigrams
----

In [32]:
def add_in_kws(file):

    df = pd.read_csv(file)
    df = gen_kw_2.add_in_kws(df)
    df.to_csv(file)
    print('add_in_kws', file)
    print('-----------------')
    return


def main_4():
    files = get_file_list_2()
    for inp in files :
        add_in_kws(inp)

main_4()

add_in_kws hs_codes_file_1_1.csv
-----------------
add_in_kws hs_codes_file_2_1.csv
-----------------
add_in_kws hs_codes_file_3_1.csv
-----------------


In [33]:
# def filter_by_hscodes(file):
#     df = pd.read_csv(file,index_col=0)
#     new_df = filter_hs_codes.filter_df(df ,level=2)
#     new_df.to_csv(file)
#     return


# def main_6():
#     global EXPERIMENT
#     files = get_file_list_2()
#     for inp in files :
#         if EXPERIMENT == False :
#             filter_by_hscodes(inp)
# main_6()  

SyntaxError: invalid syntax (<ipython-input-33-04a1e09ba2e9>, line 12)

In [5]:
import re
# Correct hs code format in hs_code_1_1
def _insert_0(match):
    txt = match.group(0)
    txt = txt.split('.')
    digit = txt[-1]
    txt[-1] = '0'+ digit
    z = '.'.join(txt)
    return z

def rectify_hs_code_format(row):
    if type(row['hs_code']) == str:
        pattern='\.[0-9]$'
        val = row['hs_code']
        res = re.sub(pattern,_insert_0,val)
        return res
    else:
        return None
    

def main_6():
    file = get_file_list_2()[0]
    df = pd.read_csv(file,index_col=0)
    try :
        del df['Unnamed: 0.1']
        del df['Unnamed: 0.1.1']
    except :
        pass
    
    df['hs_code'] = df.apply(rectify_hs_code_format,axis=1)
    df.to_csv(file)

    
main_6()
